In [2]:
import pandas as pd
import numpy as np
import tidyfinance as tf
import sqlite3

from plotnine import *
from mizani.formatters import comma_format, percent_format
from datetime import datetime

from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
load_dotenv()

True

# Parameters

In [3]:
start_date = '01/01/2000'
end_date = '12/31/2024'
db_path = '../../data/raw/crsp_monthly.sqlite'

# Load CRSP

In [4]:
def load_crsp(start_date, end_date, db_path):

    # 1. Make connection
    connection_string = (
        "postgresql+psycopg2://"
        f"{os.getenv('WRDS_USER')}:{os.getenv('WRDS_PASSWORD')}"
        "@wrds-pgdata.wharton.upenn.edu:9737/wrds"
    )

    wrds = create_engine(connection_string, pool_pre_ping=True)

    # 2. Collect dataset
    crsp_monthly = tf.download_data(
        domain="wrds",
        dataset="crsp_monthly",
        start_date=start_date,
        end_date=end_date
    )

    # 3. Preprocess

    crsp_monthly = crsp_monthly.dropna(
        subset=["ret_excess", "mktcap", "mktcap_lag"]
    )

    # 4. Save locally

    tidy_finance = sqlite3.connect(database = db_path)
    
    crsp_monthly.to_sql(
        name="crsp_monthly", 
        con=tidy_finance, 
        if_exists="replace",
        index=False
    )

    print(crsp_monthly.info())

In [5]:
load_crsp(start_date, end_date, db_path)

<class 'pandas.core.frame.DataFrame'>
Index: 1281772 entries, 1 to 1294972
Data columns (total 15 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   permno       1281772 non-null  int64         
 1   date         1281772 non-null  datetime64[ns]
 2   ret          1281772 non-null  float64       
 3   shrout       1281772 non-null  int64         
 4   altprc       1281772 non-null  float64       
 5   primaryexch  1281772 non-null  object        
 6   siccd        1281772 non-null  int64         
 7   mktcap       1281772 non-null  float64       
 8   mktcap_lag   1281772 non-null  float64       
 9   exchange     1281772 non-null  object        
 10  industry     1281772 non-null  object        
 11  mkt_excess   1281772 non-null  float64       
 12  smb          1281772 non-null  float64       
 13  hml          1281772 non-null  float64       
 14  ret_excess   1281772 non-null  float64       
dtypes: datetime64[ns](1)

# GKX Characteristics

In [6]:
pd.read_csv(
    '../../data/raw/characteristics_raw.csv',
).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4117300 entries, 0 to 4117299
Data columns (total 97 columns):
 #   Column           Dtype  
---  ------           -----  
 0   permno           int64  
 1   DATE             int64  
 2   mvel1            float64
 3   beta             float64
 4   betasq           float64
 5   chmom            float64
 6   dolvol           float64
 7   idiovol          float64
 8   indmom           float64
 9   mom1m            float64
 10  mom6m            float64
 11  mom12m           float64
 12  mom36m           float64
 13  pricedelay       float64
 14  turn             float64
 15  absacc           float64
 16  acc              float64
 17  age              float64
 18  agr              float64
 19  bm               float64
 20  bm_ia            float64
 21  cashdebt         float64
 22  cashpr           float64
 23  cfp              float64
 24  cfp_ia           float64
 25  chatoia          float64
 26  chcsho           float64
 27  chempia     

In [7]:
# Check if date 't' predicts return at 't' or 't+1'
sample = pd.read_csv('../../data/raw/characteristics_raw.csv', nrows=1000)
print(sample[['permno', 'DATE']].head())

   permno      DATE
0   10006  19570131
1   10014  19570131
2   10022  19570131
3   10030  19570131
4   10057  19570131
